In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.DataFrame()
for i in range(2010, 2019):
    tmp = pd.read_csv(f'../input/mlbplaybyplay2010s/all{i}.csv')
    tmp['YEAR'] = i
    data = data.append(tmp)
data = data.reset_index(drop=True)

In [ ]:
single, double, triple, homerun = 20, 21, 22, 23
bb_hbp = [14, 15, 16]

In [ ]:
bat_event_counts = data.groupby(['BAT_ID', 'YEAR'])['EVENT_CD'].value_counts()
pit_event_counts = data.groupby(['PIT_ID', 'YEAR'])['EVENT_CD'].value_counts()

In [68]:
bat_info = {}
for bat_id in bat_event_counts.index.get_level_values(0).unique():
    bat_info[bat_id] = {}
    for year in bat_event_counts[bat_id].index.get_level_values(0).unique():
        bat_info[bat_id][year] = {'pa': 0, 'single': 0, 'double': 0, 'triple': 0, 'homerun': 0, 'bb+hbp': 0}

        bat_info[bat_id][year]['pa'] = bat_event_counts[bat_id][year].sum()
        
        if single in bat_event_counts[bat_id][year].index:
            bat_info[bat_id][year]['single'] = bat_event_counts[bat_id][year][single].sum()
            
        if double in bat_event_counts[bat_id][year].index:
            bat_info[bat_id][year]['double'] = bat_event_counts[bat_id][year][double].sum()
            
        if triple in bat_event_counts[bat_id][year].index:
            bat_info[bat_id][year]['triple'] = bat_event_counts[bat_id][year][triple].sum()
            
        if homerun in bat_event_counts[bat_id][year].index:
            bat_info[bat_id][year]['homerun'] = bat_event_counts[bat_id][year][homerun].sum()
        
        existing_events = bat_event_counts[bat_id][year].index.intersection(bb_hbp)
        bat_info[bat_id][year]['bb+hbp'] = bat_event_counts[bat_id][year][existing_events].sum()

In [69]:
pit_info = {}
for pit_id in pit_event_counts.index.get_level_values(0).unique():
    pit_info[pit_id] = {}
    for year in pit_event_counts[pit_id].index.get_level_values(0).unique():
        pit_info[pit_id][year] = {'pa': 0, 'single': 0, 'double': 0, 'triple': 0, 'homerun': 0, 'bb+hbp': 0}
        
        pit_info[pit_id][year]['pa'] = pit_event_counts[pit_id][year].sum()
        
        if single in pit_event_counts[pit_id][year].index:
            pit_info[pit_id][year]['single'] = pit_event_counts[pit_id][year][single].sum()
            
        if double in pit_event_counts[pit_id][year].index:
            pit_info[pit_id][year]['double'] = pit_event_counts[pit_id][year][double].sum()
            
        if triple in pit_event_counts[pit_id][year].index:
            pit_info[pit_id][year]['triple'] = pit_event_counts[pit_id][year][triple].sum()
            
        if homerun in pit_event_counts[pit_id][year].index:
            pit_info[pit_id][year]['homerun'] = pit_event_counts[pit_id][year][homerun].sum()
        
        existing_events = pit_event_counts[pit_id][year].index.intersection(bb_hbp)
        pit_info[pit_id][year]['bb+hbp'] = pit_event_counts[pit_id][year][existing_events].sum()

In [84]:
%%time
bat_features = ['single/pa', 'double/pa', 'triple/pa', 'homerun/pa', '(bb+hbp)/pa', 'pa']
pit_features = ['single/pa', 'double/pa', 'triple/pa', 'homerun/pa', '(bb+hbp)/pa', 'pa']

data_dict = {}
data_dict[('is_home',)] = []
for i in range(9):
    for feature in bat_features:
        data_dict[(f'bat_{i+1}', feature)] = []
for feature in pit_features:
    data_dict[('starting_pit', feature)] = []
data_dict[('score',)] = []

game_end_indices = data.index[data['GAME_END_FL'] == 'T'].tolist()

start_index = 0
for end_index in game_end_indices:
    this_game = data.iloc[start_index:end_index+1]

    away_starting_bats = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 0].iloc[:9])
    home_starting_bats = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 1].iloc[:9])
    away_starting_pit = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 1].iloc[0]
    home_starting_pit = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 0].iloc[0]
    year = this_game.iloc[0]['YEAR']

    # away batters and home pitcher.
    data_dict[('is_home',)].append(0)
    for i in range(9):
        bat = away_starting_bats[i]
        pa = bat_info[bat][year]['pa']
        data_dict[(f'bat_{i+1}', 'single/pa')].append(bat_info[bat][year]['single'] / pa)
        data_dict[(f'bat_{i+1}', 'double/pa')].append(bat_info[bat][year]['double'] / pa)
        data_dict[(f'bat_{i+1}', 'triple/pa')].append(bat_info[bat][year]['triple'] / pa)
        data_dict[(f'bat_{i+1}', 'homerun/pa')].append(bat_info[bat][year]['homerun'] / pa)
        data_dict[(f'bat_{i+1}', '(bb+hbp)/pa')].append(bat_info[bat][year]['bb+hbp'] / pa)
        data_dict[(f'bat_{i+1}', 'pa')].append(pa)
    pit = home_starting_pit
    pa = pit_info[pit][year]['pa']
    data_dict[('starting_pit', 'single/pa')].append(pit_info[pit][year]['single'] / pa)
    data_dict[('starting_pit', 'double/pa')].append(pit_info[pit][year]['double'] / pa)
    data_dict[('starting_pit', 'triple/pa')].append(pit_info[pit][year]['triple'] / pa)
    data_dict[('starting_pit', 'homerun/pa')].append(pit_info[pit][year]['homerun'] / pa)
    data_dict[('starting_pit', '(bb+hbp)/pa')].append(pit_info[pit][year]['bb+hbp'] / pa)
    data_dict[('starting_pit', 'pa')].append(pa)
    data_dict[('score',)].append(this_game['AWAY_SCORE_CT'].iloc[-1])

    # home batters and away pitcher.
    data_dict[('is_home',)].append(1)
    for i in range(9):
        bat = home_starting_bats[i]
        pa = bat_info[bat][year]['pa']
        data_dict[(f'bat_{i+1}', 'single/pa')].append(bat_info[bat][year]['single'] / pa)
        data_dict[(f'bat_{i+1}', 'double/pa')].append(bat_info[bat][year]['double'] / pa)
        data_dict[(f'bat_{i+1}', 'triple/pa')].append(bat_info[bat][year]['triple'] / pa)
        data_dict[(f'bat_{i+1}', 'homerun/pa')].append(bat_info[bat][year]['homerun'] / pa)
        data_dict[(f'bat_{i+1}', '(bb+hbp)/pa')].append(bat_info[bat][year]['bb+hbp'] / pa)
        data_dict[(f'bat_{i+1}', 'pa')].append(pa)
    pit = away_starting_pit
    pa = pit_info[pit][year]['pa']
    data_dict[('starting_pit', 'single/pa')].append(pit_info[pit][year]['single'] / pa)
    data_dict[('starting_pit', 'double/pa')].append(pit_info[pit][year]['double'] / pa)
    data_dict[('starting_pit', 'triple/pa')].append(pit_info[pit][year]['triple'] / pa)
    data_dict[('starting_pit', 'homerun/pa')].append(pit_info[pit][year]['homerun'] / pa)
    data_dict[('starting_pit', '(bb+hbp)/pa')].append(pit_info[pit][year]['bb+hbp'] / pa)
    data_dict[('starting_pit', 'pa')].append(pa)
    data_dict[('score',)].append(this_game['HOME_SCORE_CT'].iloc[-1])
    
    start_index = end_index + 1

df = pd.DataFrame(data_dict)

Wall time: 1min 10s


In [85]:
df

is_home     bat_1                                                  \
          NaN single/pa double/pa triple/pa homerun/pa (bb+hbp)/pa   pa   
0           0  0.178918  0.033287  0.013870   0.004161    0.088766  721   
1           1  0.180301  0.030050  0.006678   0.008347    0.070117  599   
2           0  0.178918  0.033287  0.013870   0.004161    0.088766  721   
3           1  0.180301  0.030050  0.006678   0.008347    0.070117  599   
4           0  0.178918  0.033287  0.013870   0.004161    0.088766  721   
...       ...       ...       ...       ...        ...         ...  ...   
43731       1  0.192000  0.048000  0.002667   0.013333    0.130667  375   
43732       0  0.141176  0.029412  0.011765   0.026471    0.058824  340   
43733       1  0.192000  0.048000  0.002667   0.013333    0.130667  375   
43734       0  0.141176  0.029412  0.011765   0.026471    0.058824  340   
43735       1  0.144928  0.043478  0.014493   0.043478    0.086957   69   

          bat_2                      ...      bat_9                   \
      single/pa double/pa triple/pa  ... homerun/pa (bb+hbp)/pa   pa   
0      0.166102  0.040678  0.008475  ...   0.003367    0.097643  297   
1      0.120863  0.058993  0.001439  ...   0.016129    0.032258  248   
2      0.166102  0.040678  0.008475  ...   0.003367    0.097643  297   
3      0.120863  0.058993  0.001439  ...   0.013043    0.030435  230   
4      0.166102  0.040678  0.008475  ...   0.003367    0.097643  297   
...         ...       ...       ...  ...        ...         ...  ...   
43731  0.167320  0.035294  0.007843  ...   0.000000    0.037736   53   
43732  0.183333  0.024074  0.000000  ...   0.000000    0.037736  159   
43733  0.167320  0.035294  0.007843  ...   0.014553    0.085239  481   
43734  0.163504  0.049635  0.005839  ...   0.000000    0.037736  159   
43735  0.167320  0.035294  0.007843  ...   0.012987    0.110390  154   

      starting_pit                                                  score  
         single/pa double/pa triple/pa homerun/pa (bb+hbp)/pa    pa   NaN  
0         0.129649  0.042508  0.002125   0.024442    0.057386   941     3  
1         0.146703  0.065949  0.006729   0.030956    0.065949   743     6  
2         0.170330  0.051648  0.005495   0.027473    0.075824   910     5  
3         0.178470  0.052408  0.008499   0.035411    0.062323   706     3  
4         0.141575  0.047856  0.003988   0.026919    0.084746  1003     4  
...            ...       ...       ...        ...         ...   ...   ...  
43731     0.093333  0.053333  0.000000   0.020000    0.166667   150     7  
43732     0.097950  0.041002  0.005695   0.026196    0.071754   878     4  
43733     0.113924  0.063291  0.000000   0.025316    0.189873    79     9  
43734     0.055556  0.055556  0.000000   0.055556    0.138889    36     3  
43735     0.137221  0.053173  0.001715   0.032590    0.082333   583     9  

[43736 rows x 62 columns]

In [96]:
import xgboost as xgb
dmatrix = xgb.DMatrix(data=df.drop('score', axis=1), label=df['score'])
params = {'objective': 'reg:squarederror'}
cv_results = xgb.cv(dtrain=dmatrix,
                    params=params,
                    nfold=5,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    metrics="rmse",
                    as_pandas=True,
                    seed=123)
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,4.017311,0.007555,4.031612,0.039245
1,3.515741,0.007103,3.547259,0.037547
2,3.231915,0.005969,3.284249,0.033856
3,3.074128,0.006099,3.146482,0.031474
4,2.984533,0.005037,3.077335,0.028354
5,2.931686,0.005342,3.041849,0.026012
6,2.896704,0.004729,3.023101,0.023800
7,2.871554,0.004660,3.015418,0.022111
8,2.852343,0.003729,3.009837,0.021587
9,2.836832,0.002802,3.007630,0.020269


In [91]:
np.std(df['score'])

NaN    3.068503
dtype: float64

In [92]:
np.mean(df['score'])

NaN    4.280021
dtype: float64

In [94]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(df['score'], np.full(len(df['score']), np.mean(df['score']))))

In [95]:
rms

3.068503226933378